<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_01_Arboles/01_%C3%81rboles_Decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRO: Análisis Univariante 
Explicación de cómo realizar análisis univariantes.

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/16tnAAaiT6mrelG8zRg26crXAChWmzjOG2NyA0oyh4X0/edit?usp=sharing)
*   [Enlace a este notebook en Colab](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/03-analisis-univariante-causal-multivariante/03_A___Analisis_univariante.ipynb)




## 0101 Qué es un árbol de decisión?
Un tipo de algoritmo de aprendizaje supervisado que se basa en realizar particiones a partir de distintos niveles de las variables disponibles.

## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 0103 Carga el dataset de accidentes cardiacos
Para probar a hacer árboles de decisión utilizaremos un dataset sobre accidentes coronarios que he encontrado en Kaggle ([aquí](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data)). 

Podéis encontrar el archivo listo para importar en mi github: 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/heart_failure.csv'. 

Importa este dataset en un dataframe llamado **df**.

In [15]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/heart_failure.csv'

# Importa csv
df = pd.read_csv(url)

# Visualización primeras filas
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


# 02 EDA
Realizaremos un pequeño análisis exploratorio visual para familiarizarnos con el dataset. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


# 03 Construcción árbol de decisión a mano
Nunca lo haremos en la vida real, pero para afianzar el entendimiento de este tipo de algoritmos, vamos a reproducir el proceso de entrenamiento a mano.

## 0301 DataFrame
Para simplificar, haremos esto con un dataframe consistente solo en las columnas ['age', 'anaemia', 'serum_creatinine', 'DEATH_EVENT'] renombradas como ['Edad', 'Anemia', 'Serum', 'Fallece'].

In [17]:
df_resumen = df[['age', 'anaemia', 'serum_creatinine', 'DEATH_EVENT']]
df_resumen.columns = ['Edad', 'Anemia', 'Serum', 'Fallece']
df_resumen.sample(10).reset_index(drop=True)

,Edad,Anemia,Serum,Fallece
0,50.000,1,1.2,0
1,65.000,1,0.8,0
2,60.000,0,0.6,1
3,72.000,1,1.7,1
4,52.000,1,1.0,0
5,50.000,0,1.6,0
6,65.000,1,1.1,0
7,70.000,0,0.8,0
8,60.000,1,1.0,0
9,60.667,1,1.5,1


## 0301 Seleccionamos el nodo raíz
Para seleccionar el nodo raíz generaremos árboles muy sencillos, de solo 3 nodos (una sola partición).

### 030101 Variable categórica
Probamos a generar un árbol de sólo dos particiones.

In [13]:
df_i = df_resumen.groupby('Anemia')['Fallece'].agg(['count', 'sum']).reset_index()
df_i.columns = ['Anemia', 'Total', 'Fallece = 1']
df_i['Fallece = 0'] = df_i['Total'] - df_i['Fallece = 1']
df_i['Mean'] = np.round(df_i['Fallece = 1']/df_i['Total'], 4)
df_i

,Anemia,Total,Fallece = 1,Fallece = 0,Mean
0,0,170,50,120,0.2941
1,1,129,46,83,0.3566


### 030102 Variable Continua
Las variables continuas pueden dar lugar a diversos árboles.

In [22]:
df_i = df_resumen.copy(deep=True)

filtro_i = df_i['Anemia'] == 1
nombre_filtro_True = 'Anemia = 1'
nombre_filtro_Falso = 'Anemia = 0'

df_i['Filtro_i'] = filtro_i.replace({True: nombre_filtro_True, False: nombre_filtro_Falso})
df_i = df_i.groupby('Filtro_i')['Fallece'].agg(['count', 'sum']).reset_index()
df_i.columns = ['Filtro_i', 'Total', 'Fallece = 1']
df_i['Fallece = 0'] = df_i['Total'] - df_i['Fallece = 1']
df_i['Mean'] = np.round(df_i['Fallece = 1']/df_i['Total'], 4)
df_i

,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Anemia = 0,170,50,120,0.2941
1,Anemia = 1,129,46,83,0.3566


In [23]:
df_i = df_resumen.copy(deep=True)
var = 'Edad'
valor = df_i[var].quantile(0.5)

filtro_i = df_i[var] >= valor
nombre_filtro_True = str(var) + ' >= ' + str(round(valor,2))
nombre_filtro_Falso = str(var) + ' < ' + str(round(valor,2))

df_i['Filtro_i'] = filtro_i.replace({True: nombre_filtro_True, False: nombre_filtro_Falso})
df_i = df_i.groupby('Filtro_i')['Fallece'].agg(['count', 'sum']).reset_index()
df_i.columns = ['Filtro_i', 'Total', 'Fallece = 1']
df_i['Fallece = 0'] = df_i['Total'] - df_i['Fallece = 1']
df_i['Mean'] = np.round(df_i['Fallece = 1']/df_i['Total'], 4)
df_i

,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Edad < 60.0,129,31,98,0.2403
1,Edad >= 60.0,170,65,105,0.3824


In [35]:
var = 'Edad'
valores_unicos = sorted(df_resumen[var].unique())

for i, valor in enumerate(valores_unicos[1:]):
  df_i = df_resumen.copy(deep=True)
  valor = 0.5*valor + 0.5*valores_unicos[i]
  print(valor)
  
  filtro_i = df_i[var] >= valor
  nombre_filtro_True = str(var) + ' >= ' + str(round(valor,2))
  nombre_filtro_Falso = str(var) + ' < ' + str(round(valor,2))

  df_i['Filtro_i'] = filtro_i.replace({True: nombre_filtro_True, False: nombre_filtro_Falso})
  df_i = df_i.groupby('Filtro_i')['Fallece'].agg(['count', 'sum']).reset_index()
  df_i.columns = ['Filtro_i', 'Total', 'Fallece = 1']
  df_i['Fallece = 0'] = df_i['Total'] - df_i['Fallece = 1']
  df_i['Mean'] = np.round(df_i['Fallece = 1']/df_i['Total'], 4)
  display(df_i)

40.5


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Edad < 40.5,7,0,7,0.0000
1,Edad >= 40.5,292,96,196,0.3288


41.5


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Edad < 41.5,8,0,8,0.0000
1,Edad >= 41.5,291,96,195,0.3299


42.5


...

,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Edad < 88.5,293,91,202,0.3106
1,Edad >= 88.5,6,5,1,0.8333


92.0


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Edad < 92.0,296,93,203,0.3142
1,Edad >= 92.0,3,3,0,1.0000


94.5


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Edad < 94.5,297,94,203,0.3165
1,Edad >= 94.5,2,2,0,1.0000


In [36]:
var = 'Serum'
valores_unicos = sorted(df_resumen[var].unique())

for i, valor in enumerate(valores_unicos[1:]):
  df_i = df_resumen.copy(deep=True)
  valor = 0.5*valor + 0.5*valores_unicos[i]
  print(valor)
  
  filtro_i = df_i[var] >= valor
  nombre_filtro_True = str(var) + ' >= ' + str(round(valor,2))
  nombre_filtro_Falso = str(var) + ' < ' + str(round(valor,2))

  df_i['Filtro_i'] = filtro_i.replace({True: nombre_filtro_True, False: nombre_filtro_Falso})
  df_i = df_i.groupby('Filtro_i')['Fallece'].agg(['count', 'sum']).reset_index()
  df_i.columns = ['Filtro_i', 'Total', 'Fallece = 1']
  df_i['Fallece = 0'] = df_i['Total'] - df_i['Fallece = 1']
  df_i['Mean'] = np.round(df_i['Fallece = 1']/df_i['Total'], 4)
  display(df_i)

0.55


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Serum < 0.55,1,0,1,0.0000
1,Serum >= 0.55,298,96,202,0.3221


0.6499999999999999


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Serum < 0.65,5,2,3,0.4000
1,Serum >= 0.65,294,94,200,0.3197


0.725


...

,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Serum < 6.45,296,93,203,0.3142
1,Serum >= 6.45,3,3,0,1.0000


7.9


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Serum < 7.9,297,94,203,0.3165
1,Serum >= 7.9,2,2,0,1.0000


9.2


,Filtro_i,Total,Fallece = 1,Fallece = 0,Mean
0,Serum < 9.2,298,95,203,0.3188
1,Serum >= 9.2,1,1,0,1.0000
